## Load wells from CSV save to SHP

Some preliminaries...

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import welly
welly.__version__

'0.1.1'

In [2]:
import os
env = %env

## Load wells from CSV

This does not do a lot yet. This is the first step in trying to make a CSV constructor.

It may not be possible to generalize this in a sensible way.

In [3]:
import csv
fname = 'data/Nova_Scotia_wells_2015.csv'
reader = csv.DictReader(open(fname))

wells = []

for line in reader:
    params = {'header': {},
              'location': {},
             }
    
    for key, value in line.items():
        sect, item = key.split('.')
        params[sect][item] = value
        
    # Deal with surface location nonsense.
    
    # First the numerics.
    try:
        lat = float(params['location'].get('latitude'))
        lon = float(params['location'].get('longitude'))
    except:
        lat, lon = 0, 0
        
    try:
        x = float(params['location'].get('x'))
        y = float(params['location'].get('y'))
    except:
        x, y = 0, 0

    # Then the strings.
    datum = params['location'].get('datum')  # Empty string if missing
    utm = params['location'].get('datum')    # Empty string if missing
    
    # STRICT RULES
    # Either a well location has what it needs, or it doesn't. 
    
    #     # If we have (lat, lon) then chuck everything else out.
    #     if lat and lon:
    #         del params['location']['x']
    #         del params['location']['y']
    #         del params['location']['utm']
    #         if not datum:
    #             del params['location']['latitude']
    #             del params['location']['longitude']

    #     # Otherwise, keep fully qualified (x, y)s
    #     else:
    #         del params['location']['latitude']
    #         del params['location']['longitude']
    #         if (not x and y) or (not datum) or (not utm):
    #             del params['location']['x']
    #             del params['location']['y']
    #             del params['location']['datum']
    #             del params['location']['utm']

    # Build the well and add it to the list.
    h = welly.Header(params['header'])
    l = welly.Location(params['location'])
    w = welly.Well({'header': h, 'location': l})
    wells.append(w)

Get the index of the well P-129 in this list:

In [4]:
[w.header.license for w in wells].index('P-129')

10

Add curves to this well:

In [5]:
wells[10]

In [86]:
wells[10].location.basin


Location({'basin': 'Windsor Basin', 'td': 1935.0, 'longitude': 63.75679444, 'gl': 90.3, 'position': None, 'crs': CRS({'init': 'epsg:4269', 'no_defs': True}), 'deviation': None, 'kb': 94.8, 'latitude': 45.20951028})

In [6]:
wells[10].add_curves_from_las('data/las/P-129_out.LAS')

Add a CRS:

In [52]:
wells[10].location.crs_from_epsg(4269)  # NAD83

Check everything:

In [66]:
wells[0].location

Location({'y': 529556.93, 'deviation': None, 'td': 1259.0, 'longitude': 62.62126806, 'x': 5045389.81, 'position': None, 'crs': CRS({}), 'utm': 20.0, 'kb': 44.68, 'latitude': 45.56141944})

## Save well to SHP

We'll make a loop to do many wells, but we only have proper info for one:

In [67]:
wells_to_export = wells  # Most of these will fail, because the spreadsheet isn't strictly complete

In [68]:
len(wells_to_export)

151

In [69]:
my_crs = wells[10].location.crs.data  # Most of these will fail.

To work with shapefiles, we need to install the *fiona* and *shapely* Python modules. They aren't dependencies for `welly` or `striplog`, but they are required to continue with this workflow.

In [70]:
import fiona
from fiona import crs
from shapely.geometry import Point, LineString, mapping

In [90]:
point_schema = {'geometry': 'Point',
                'properties': {'license': 'str',
                               'name': 'str',
                               'uwi': 'str',
                               'td': 'int',
                               #'gr': 'float'
                               }
                }

fname = 'data/shapefiles/well.shp'

with fiona.open(fname, "w",
                driver="ESRI Shapefile",
                crs=my_crs,
                schema=point_schema) as f:

    count = 0
    for w in wells_to_export:
        try:
            if w.location.latitude and w.location.longitude != 0:
                
                if w.location.basin == 'Windsor Basin':
                    p = Point(w.location.latitude, -w.location.longitude)
                    f.write({'geometry': mapping(p),
                             'properties': {'license': w.header.license,
                                            'name': w.header.name,
                                            'uwi': w.uwi,
                                            'td': w.location.td,
                                            #'gr': float(w.data['GR'].mean())
                                           }
                             })
                    count += 1
        except:
            pass
print('Exported {} well(s)'.format(count))

Exported 14 well(s)


Check it:

In [91]:
import pprint
with fiona.open(fname, "r", driver="ESRI Shapefile") as c:
    for i in c:
        pprint.pprint(i)

{'geometry': {'coordinates': (45.29485083, -63.72693361), 'type': 'Point'},
 'id': '0',
 'properties': OrderedDict([('license', 'P-133'),
                            ('name', 'E-38-A/11-E-5'),
                            ('uwi', None),
                            ('td', 1726)]),
 'type': 'Feature'}
{'geometry': {'coordinates': (45.27526, -63.72334833), 'type': 'Point'},
 'id': '1',
 'properties': OrderedDict([('license', 'P-130'),
                            ('name', 'N-14-A/11-E-5'),
                            ('uwi', None),
                            ('td', 2618)]),
 'type': 'Feature'}
{'geometry': {'coordinates': (45.20951028, -63.75679444), 'type': 'Point'},
 'id': '2',
 'properties': OrderedDict([('license', 'P-129'),
                            ('name', 'Kennetcook #2'),
                            ('uwi', None),
                            ('td', 1935)]),
 'type': 'Feature'}
{'geometry': {'coordinates': (45.19742861, -63.71606389), 'type': 'Point'},
 'id': '3',
 'properties': 